In [ ]:
%pip install --upgrade --quiet  langchain langchain-community langchainhub langchain-chroma bs4
# pip install -qU langchain-openai

In [ ]:
from langchain_community.llms import Ollama
llm = Ollama(model="llamafamily/llama3-chinese-8b-instruct",temperature=0.1,top_p=0.3)
# llm.invoke("台灣是中國的嗎?")

In [1]:
import os
from langchain_community.llms import Ollama
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_community.embeddings import OllamaEmbeddings 
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

model_name = "llamafamily/llama3-chinese-8b-instruct"
file_path = "C:\\Users\\User\\Documents\\work\\tcm-agent\\data\\txt"

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use five sentences maximum and keep the "
    "answer concise. Answer in Chinese."
    "\n\n"
    "{context}"
)

llm = Ollama(model=model_name,temperature=0.2,top_p=0.3)
embedding= OllamaEmbeddings(model=model_name,temperature=0.2,top_p=0.3)


class RagChain:
    def __init__(self,llm, embedding,file_name,system_prompt) -> None:
        self.llm = llm
        self.embedding = embedding
        # self.system_prompt = system_prompt
        
        total_path = os.path.normpath(f"{file_path}/{file_name}")
        self.loader = TextLoader(total_path,encoding='utf-8')
        self.docs = self.loader.load()
        
        self.text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
        self.splits = self.text_splitter.split_documents(self.docs)
        self.vector_store = Chroma.from_documents(documents= self.splits, embedding= embedding)
        self.retriever = self.vector_store.as_retriever()
        
        self.prompt = ChatPromptTemplate.from_messages(
            [
                ("system", system_prompt),
                ("human", "{input}"),
            ]
        )
        
        self.question_answer_chain = create_stuff_documents_chain(self.llm, self.prompt)
        self.rag_chain = create_retrieval_chain(self.retriever, self.question_answer_chain)
            
    def __call__(self, input_data):
        return self.rag_chain.invoke(input_data)

    
# 中醫十問
zhongyi_shiwen = RagChain(llm, embedding, "中醫十問.txt", system_prompt)   
# 九大體質
jiuda_tizhi = RagChain(llm, embedding, "九大體質.txt", system_prompt) 
self_introduction = RagChain(llm, embedding, "自我介紹.txt", system_prompt)    
SongBieYu = RagChain(llm, embedding, "送別語.txt", system_prompt)  

In [10]:
response_4 = SongBieYu({"input": "based on knowledge。參考文件內容說再見，完整一點的版本"})
print(response_4['answer'])

很高興為您服務，祝平安喜樂，再見。


In [4]:
response_1 = self_introduction({"input": "based on knowledge。請按照＂自我介紹.txt＂文件做自我介紹"})
print(response_1['answer'])

您好，我是一名中醫師，專門提供中醫治療方案和飲食建議。我會根據您的回答，開出適合的處方。在治療過程中，身體可能會出現一些反應，這是正常的，請不用擔心。記住，中醫治病是根據身體的症狀，而非西醫的檢驗報告，所以請詳細描述您的症狀，這樣才能幫助我們為您提供最好的治療。祝平安喜樂！


In [2]:

response_2 = zhongyi_shiwen({"input": "based on knowledge。請分析文件，中醫十問是哪十問?請告訴我問診名稱就好。"})
print(response_2['answer'])


中醫十問分成：睡眠、胃口、大便、小便、口渴、寒熱、汗、體力、性功能和女子月經。


In [3]:


response_3 = jiuda_tizhi({"input": "based on knowledge。請分析文件中有提到以下描述嗎?如果沒有請回答不知道，如果有，這是哪一種體質?\n- - -\n常見表現特徵為手腳冰冷、臉色蒼白、身體畏寒、喜歡吃溫熱飲食。容易發生水腫、腹瀉大便不成形，易表現出精神不濟，容易感冒。體態上呈現肌肉鬆軟不結實，舌頭胖大、舌苔變厚呈白色水滑。"})
print(response_3['answer'])


陽虛質


In [11]:
response_4 = jiuda_tizhi({"input": "based on knowledge。如果沒有請回答不知道，如果有，請詳細說明以下體質有什麼病症和飲食建議\n- - -\n血瘀質"})
print(response_4['answer'])


血瘀質的人容易出現瘀斑、膚色黯沉、碰撞容易有瘀青、身體痠痛的狀況，適合吃活血的食物，例如薑黃、紅麴、納豆、酒釀、丹參、山楂、玫瑰花、茉莉花等，減少燒、烤、炸、辣、甜食、及油膩食物。


In [12]:

response_5 = zhongyi_shiwen({"input": "based on knowledge。中醫十問當中的第一問的問診細節是什麼?只要問診細節：後面的"})
print(response_5['answer'])


你的睡眠如何？是否一覺到天亮？是否每天定時會醒？如果會醒，是幾點會醒？是否多夢？等等。


In [13]:
tttt = llm.invoke(f"請把以下兩句話請按照順序進行合併。第一句：{response_1['answer']}\n 接著讓我們來開始第一個問題 \n第二句：{response_5['answer']}")
print(tttt)

您好，我是一名中醫師，專門提供中醫治療方案和飲食建議。我會根據您的回答，開出適合的處方。在治療過程中，身體可能會出現一些反應，這是正常的，請不用擔心。記住，中醫治病是根據身體的症狀，而非西醫的檢驗報告，所以請詳細描述您的症狀，這樣才能幫助我們為您提供最好的治療。祝平安喜樂！讓我們來開始第一個問題：你的睡眠如何？是否一覺到天亮？是否每天定時會醒？如果會醒，是幾點會醒？是否多夢？等等。


In [14]:
import json

class Agent:
    sex = 'male'
    messages = []
    _qa = ["睡眠","胃口","大便","小便","口渴","寒熱","汗","體力","性功能","女子月經"]
    questions = {}
    answers = {}
    question_count = 0
    def __init__(self,sex = 'male') -> None:
        self.sex = sex
        pass
    
    def invoke(self,user_input):
        if self.question_count == 0:
            self_introduction_response = self_introduction({"input": "based on knowledge。請按照＂自我介紹.txt＂文件做自我介紹"})
            zhongyi_shiwen_response = zhongyi_shiwen({"input": "based on knowledge。中醫十問當中的第一問的問診細節是什麼?只要問診細節：後面的文字就好"})
            question_key = self._qa[self.question_count]
            self.questions[question_key] = zhongyi_shiwen_response['answer']
            
            response = llm.invoke(f"請把以下兩句話請按照順序進行合併。第一句：{self_introduction_response['answer']}\n 接著讓我們來開始第一個問題 \n第二句：{zhongyi_shiwen_response['answer']}")
        elif (self.sex == 'male' and self.question_count >= 9) or (self.sex == 'female' and self.question_count >= 10 ):

            # 將過濾後的字典轉換為 JSON 字串
            # json_string = json.dumps(self.answers, ensure_ascii=False, indent=4)
            formatted_list = [f"問{key}，答 {value}" for key, value in self.answers.items()]
            formatted_string = "。".join(formatted_list)
            # llm_merge_response =  llm.invoke(f"請把以資料請按照順序進行合併，並按照格式表示＂問key，答value＂。以下是資料：\n{json_string}")
            jiuda_tizhi_response = jiuda_tizhi({"input": f"based on knowledge。請分析文件，以下描述最接近哪幾種體質特徵(1~3種)?有什麼飲食上的建議?\n- - -\n{formatted_string}"})
            # llm_organizing_response =  llm.invoke(f"請把以資料重新整理(先描述體質種類，最後才是給飲食建議)、合併重複內容。以下是資料：\n{jiuda_tizhi_response['answer']}")
            response = jiuda_tizhi_response['answer']
        else:
            answer_key = self._qa[self.question_count-1]
            self.answers[answer_key] = user_input
            
            key = self._qa[self.question_count]
            zhongyi_shiwen_response = zhongyi_shiwen({"input": f"based on knowledge。中醫十問當中的{key}的問診細節是什麼?只要問診細節：後面的的文字就好"})
            
            question_key = self._qa[self.question_count]
            self.questions[question_key] = zhongyi_shiwen_response['answer']
            
            llm_response =  llm.invoke(f"如果以下文字內容是正面的，請你回答＂好的，我知道了！＂；如果以下文字內容是負面的，請總結以下文字大概意思就好字數不超過二十字，並在最前面加上＂我了解你有(_總結後的文字_)的問題。＂。\n{user_input}")
            response = llm.invoke(f"請把以下幾段話請按照順序進行合併。 {llm_response} \n　接著讓我們來進行下一個問題 \n {zhongyi_shiwen_response['answer']}")
            
        
        
        self.question_count += 1
        self.messages.append(response)
        print(response)
        return response

In [15]:
agent = Agent(sex='female')
agent.invoke('我要開始問診')
agent.invoke('我可以正常入睡，但半夜常常會莫名起來，然後就睡不回去了')
agent.invoke('我最近都沒什麼胃口吃不下飯，只想吃甜食和炸物')
agent.invoke('最近腹瀉吃什麼馬上就會拉肚子')
agent.invoke('小便是深黃色，大概間隔四五個小時久久才會去尿一次')
agent.invoke('喜歡喝冰水，但是會因為忙到忘記喝水，一天可能才喝1200c.c.的水量')
agent.invoke('覺得四肢冰冷')
agent.invoke('半夜會盜汗')
agent.invoke('一直覺得很疲憊，怎麼睡都不夠，早上都爬不起來')
agent.invoke('性功能一切正常')
agent.invoke('月經不規律，每一次來都很痛，血算是偏多')

您好，我是一名中醫師，專門提供中醫治療方案和飲食建議。我會根據您的回答，開出適合的處方。在治療過程中，身體可能會出現一些反應，這是正常的，請不用擔心。記住，中醫治病是根據身體的症狀，而非西醫的檢驗報告，所以請詳細描述您的症狀，這樣才能幫助我們為您提供最好的治療。祝平安喜樂！讓我們來開始第一個問題：你的睡眠如何？是否一覺到天亮？是否每天定時會醒？如果會醒，是幾點會醒？是否多夢？等等。
我了解你有入睡困難的問題。接著讓我們來進行下一個問題，你感覺餓嗎？有欲望想吃什麼特別的食物或是喜愛什麼味道的食物？或是不餓，完全沒有胃口。
我了解你有消化不良的問題。接著，讓我們來進行下一個問題：你的大便是否正常？每天有否大便？大便顏色是什麼？是否下利？很臭還是無味？等等。
我了解你有腹瀉的問題。接著讓我們來進行下一個問題：你的小便是什麼顏色？頻尿嗎？還是尿不出來？還是沒有尿意？平均一天幾次？等等。
我了解你有尿頻率不穩定的問題。接著讓我們來進行下一個問題：你很渴嗎？如渴，請問最想喝什麼溫度的水？如不渴，是否時常會忘記喝水？還是再怎麼喝也不能止渴呢？
寒熱，請問你平時覺得身體很熱還是很冷？手腳冰冷嗎？我了解你有忘記喝水的問題。接著讓我們來進行下一個問題。
我了解你有寒冷感覺的問題。接著讓我們來進行下一個問題：你容易出汗嗎？會半夜盜汗嗎？會時常流汗不止嗎？還是不出汗的身體呢？
我了解你有睡眠障礙的問題。接著讓我們來進行下一個問題：你的體力如何？精神好嗎？還是一直疲憊中？早上起床時，是否精神奕奕呢？是否無法起床呢？是否精神能夠集中呢？
我了解你有疲憊和睡眠不足的問題。接著，請問你的性功能如何？是否存在任何困難或問題？如果是，我們可以進一步探討可能的原因和解決方案。
我了解你有正常性的問題。接著讓我們來進行下一個問題：女子月經。無論妳有無月經，都要詳細說明妳的月經情形，是否延後還是每次都提前呢？痛不痛呢？生過小孩嗎？
根據您提供的信息，您可能屬於血瘀質或氣鬱質的人。對於血瘀質的人，建議多吃活血的食物，如薑黃、紅麴、納豆、酒釀、丹參、山楂、玫瑰花、茉莉花等，減少燒、烤、炸、辣、甜食、及油膩食物。對於氣鬱質的人，建議多吃可以幫助身體氣機活動的食物，如陳皮、柑橘類水果、咖哩、金針花、玫瑰花、茉莉花等，減少冰冷或寒涼的食物。另外，您也可以考慮增加運動和適當的休息，以改善體力和睡眠問題。


'根據您提供的信息，您可能屬於血瘀質或氣鬱質的人。對於血瘀質的人，建議多吃活血的食物，如薑黃、紅麴、納豆、酒釀、丹參、山楂、玫瑰花、茉莉花等，減少燒、烤、炸、辣、甜食、及油膩食物。對於氣鬱質的人，建議多吃可以幫助身體氣機活動的食物，如陳皮、柑橘類水果、咖哩、金針花、玫瑰花、茉莉花等，減少冰冷或寒涼的食物。另外，您也可以考慮增加運動和適當的休息，以改善體力和睡眠問題。'

In [16]:
agent = Agent(sex='male')
agent.invoke('我要開始問診')
agent.invoke('我因為工作的關係常常熬夜，但是可以一覺到天亮')
agent.invoke('我最近都沒什麼胃口吃不下飯')
agent.invoke('最近便祕，大概兩三天才會大出一點點，像羊便便一樣一顆一顆，又乾又硬')
agent.invoke('小便是透明的，大概半個小時就會上一次廁所')
agent.invoke('喜歡喝冰水，一天可以喝到2200c.c.的水量')
agent.invoke('覺得四肢很熱')
agent.invoke('平常待在冷氣房不怎麼流汗，但是只要一離開冷氣房就會一直流汗')
agent.invoke('一直覺得很疲憊，怎麼睡都不夠')
agent.invoke('性無能，勃起有困難')

您好，我是一名中醫師，專門提供中醫治療方案和飲食建議。我會根據您的回答，開出適合的處方。在治療過程中，身體可能會出現一些反應，這是正常的，請不用擔心。記住，中醫治病是根據身體的症狀，而非西醫的檢驗報告，所以請詳細描述您的症狀，這樣才能幫助我們為您提供最好的治療。祝平安喜樂！讓我們來開始第一個問題：你的睡眠如何？是否一覺到天亮？是否每天定時會醒？如果會醒，是幾點會醒？是否多夢？等等。
我了解你有熬夜的問題。接著讓我們來進行下一個問題，你感覺餓嗎？有欲望想吃什麼特別的食物或是喜愛什麼味道的食物？或是不餓，完全沒有胃口。
我了解你有消化不良的問題。接著讓我們來進行下一個問題：你的大便是否正常？每天有大便嗎？大便顏色是什麼？是下利嗎？很臭還是無味？等等。
我了解你有便秘的问题。接著讓我們來進行下一個問題：你的小便是什麼顏色？頻尿嗎？還是尿不出來？還是沒有尿意？平均一天幾次？等等。
我了解你有尿頻繁的問題。接著讓我們來進行下一個問題：你很渴嗎？如渴，請問最想喝什麼溫度的水？如不渴，是否時常會忘記喝水？還是再怎麼喝也不能止渴呢？
我了解你有喜歡喝冰水的問題。接著讓我們來進行下一個問題：你平時覺得身體很熱還是很冷？手腳冰冷嗎？
我了解你有體溫升高的問題。接著讓我們來進行下一個問題：你容易出汗嗎？會半夜盜汗嗎？會時常流汗不止嗎？還是不出汗的身體呢？
我了解你有“出冷氣房後會流汗”的問題。接著讓我們來進行下一個問題：體力如何？精神好嗎？還是一直疲憊中？早上起床時，是否精神奕奕呢？是否無法起床呢？是否精神能夠集中呢？
我了解你有睡眠不足的問題。接著，請問你的性功能如何？是否存在任何問題或困難？如果是，我們可以進一步探討和解決這些問題。
根據您的描述，您的體質特徵最接近血瘀質。飲食建議包括減少燒、烤、炸、辣、甜食、及油膩食物，並增加活血的食物，如薑黃、紅麴、納豆、酒釀、丹參、山楂、玫瑰花、茉莉花等。


'根據您的描述，您的體質特徵最接近血瘀質。飲食建議包括減少燒、烤、炸、辣、甜食、及油膩食物，並增加活血的食物，如薑黃、紅麴、納豆、酒釀、丹參、山楂、玫瑰花、茉莉花等。'

In [18]:
print(f"questions:\n{agent.questions}\n")
print(f"answers:\n{agent.answers}\n")
print(f"messages:\n{agent.messages}\n")

questions:
{'睡眠': '你的睡眠如何？是否一覺到天亮？是否每天定時會醒？如果會醒，是幾點會醒？是否多夢？等等。', '胃口': '你感覺餓嗎？有欲望想吃什麼特別的食物或是喜愛什麼味道的食物？或是不餓，完全沒有胃口。', '大便': '你的大便是否正常？每天有大便嗎？大便顏色是什麼？是下利嗎？很臭還是無味？等等。', '小便': '你的小便是什麼顏色？頻尿嗎？還是尿不出來？還是沒有尿意？平均一天幾次？等等。', '口渴': '你很渴嗎？如渴，最想喝什麼溫度的水？如不渴，時常會忘記喝水嗎？還是再怎麼喝也不能止渴呢？', '寒熱': '你平時覺得身體很熱還是很冷？手腳冰冷嗎？', '汗': '你容易出汗嗎？會半夜盜汗嗎？會時常流汗不止嗎？還是不出汗的身體呢？', '體力': '問診問題： 體力如何\n問診細節： 精神好嗎？還是一直疲憊中？早上起床時，是否精神奕奕呢？是否無法起床呢？是否精神能夠集中呢？', '性功能': '問診問題： 性功能\n問診細節： 你性功能好嗎？等等。', '女子月經': '問診問題：女子月經\n問診細節：無論妳有無月經，都要詳細說明妳的月經情形，是否延後還是每次都提前呢？痛不痛呢？生過小孩嗎？'}

answers:
{'睡眠': '我因為工作的關係常常熬夜，但是可以一覺到天亮', '胃口': '我最近都沒什麼胃口吃不下飯', '大便': '最近便祕，大概兩三天才會大出一點點，像羊便便一樣一顆一顆，又乾又硬', '小便': '小便是透明的，大概半個小時就會上一次廁所', '口渴': '喜歡喝冰水，一天可以喝到2200c.c.的水量', '寒熱': '覺得四肢很熱', '汗': '平常待在冷氣房不怎麼流汗，但是只要一離開冷氣房就會一直流汗', '體力': '一直覺得很疲憊，怎麼睡都不夠', '性功能': '性功能一切正常'}

messages:
['您好，我是一名中醫師，專門提供中醫治療方案和飲食建議。我會根據您的回答，開出適合的處方。在治療過程中，身體可能會出現一些反應，這是正常的，請不用擔心。記住，中醫治病是根據身體的症狀，而非西醫的檢驗報告，所以請詳細描述您的症狀，這樣才能幫助我們為您提供最好的治療。祝平安喜樂！讓我們來開始第一個問題：你的睡眠如何？是否一覺到天亮？是否每天定時會醒？如果會醒，是幾點會醒？是否多夢？等等。', 

In [ ]:
# 如果想要重新回答上一題
agent.question_count -= 1

In [17]:
print(f"agent.question_count: {agent.question_count}")

agent.question_count: 10
